# Exercise - 7
Train and fine-tune a decision tree for the moons dataset by following these steps: 

a. Use `make_moons(n_samples= 10000, noise= 0.4)` to generate a moons dataset.
<br>
b. Use `train_test_split()` to split the dataset into a training set and a test set.
<br>
c. Use grid search with cross-validation to find good hyperparameter values for a `DecisionTreeClassifier`.
<br>
d. Train it on the full training set using these hyperparameters and measure your model's performance on the test set. You should get roughly 85% to 87% accuracy.
## Imports

In [22]:
import numpy as np
from scipy.stats import mode
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone, BaseEstimator
from sklearn.metrics import accuracy_score

## Generating Dataset

In [2]:
X, y = make_moons(n_samples= 10000, noise= 0.4, random_state= 42)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42)

## First model

In [4]:
tree_clf = DecisionTreeClassifier(random_state= 42)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [5]:
tree_clf.score(X_test, y_test)

0.806

## Fine Tuning

In [6]:
param_grid: dict[str, list] = {
    'max_leaf_nodes': list(range(2, 200)),
    'max_depth': list(range(1, 7)),
    'min_samples_split': [2, 3, 4]
}
grid_cv = GridSearchCV(DecisionTreeClassifier(random_state= 42), param_grid, cv= 3)

In [7]:
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]})

In [8]:
grid_cv.best_params_

{'max_depth': 6, 'max_leaf_nodes': 13, 'min_samples_split': 2}

In [9]:
grid_cv.score(X_test, y_test)

0.8552

# Exercise - 8
Grow a forest.

a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn's ShuffleSplit class for this.
<br>
b. Train one Decision Tree on each subset, using the best hyperparameter values found above. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.
<br>
c. Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy's mode() function for this). This gives you majority-vote predictions over the test set.
<br>
d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!

In [16]:
n_trees: int = 1000

rs = ShuffleSplit(
    n_splits= n_trees, 
    train_size= 0.01,
    random_state= 42
)

In [17]:
subsets: list[tuple[np.ndarray, np.ndarray]] = []

for train_index, _ in rs.split(X_train):
    X_train_mini = X_train[train_index]
    y_train_mini = y_train[train_index]
    subsets.append((X_train_mini, y_train_mini))

In [18]:
subsets[0]

(array([[-0.47833889,  0.84508783],
        [ 1.04262131, -0.7681717 ],
        [ 0.36933415,  1.06186909],
        [-0.53611219, -0.14400454],
        [ 1.13555621, -0.17402358],
        [ 0.79958258, -0.0631054 ],
        [-1.00000369,  0.62374659],
        [ 1.2314914 , -0.96308203],
        [ 1.50572225, -0.02457978],
        [-0.15110406,  0.6509247 ],
        [-0.55945493, -0.1417014 ],
        [ 2.00671955, -0.18298219],
        [ 0.23391446,  0.97353684],
        [ 1.82142782, -0.0265036 ],
        [-0.92355848,  0.17920821],
        [ 1.29908568, -0.0559383 ],
        [-0.66625529, -0.05276538],
        [ 0.0045033 ,  0.53707192],
        [-1.05531098,  0.23439879],
        [ 0.13194117, -0.37237219],
        [-0.91259678,  0.61406954],
        [-0.01347511,  1.26927331],
        [ 2.02096953,  0.03738029],
        [ 2.02837108, -0.37526966],
        [-1.14057175, -0.33149396],
        [ 1.11126952, -0.38632803],
        [ 1.70965718, -0.43756531],
        [ 1.21180039,  0.386

In [26]:
rnd_forest: list[BaseEstimator] = [clone(grid_cv.best_estimator_) for _ in range(n_trees)]
accuracy_scores: list[float] = []

for tree, (X_sub, y_sub) in zip(rnd_forest, subsets):
    tree.fit(X_sub, y_sub)
    
    predictions = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, predictions))

In [27]:
np.mean(accuracy_scores)

0.7938184

In [31]:
y_pred = np.empty((n_trees, len(X_test)), dtype= 'uint8')

for index, tree in enumerate(rnd_forest):
    y_pred[index] = tree.predict(X_test)

In [33]:
mode_pred, count = mode(y_pred)

In [34]:
accuracy_score(y_test, mode_pred.reshape(-1))

0.8644